In [12]:
# Import libraries
import pandas as pd
import os
import requests

In [3]:
BASE_API = "https://www.imf.org/external/datamapper/api/v1/"

In [4]:
# Code of the indicator that is required
# collecting data for GDP, current prices - NGDPD
indicator = "NGDPD"

For more indicator names of refer this
https://www.imf.org/external/datamapper/api/v1/indicators

Source: IMF
https://www.imf.org/external/datamapper/api/help

In [10]:
years_list = list(range(2011,2026))
years = ",".join(map(str,years_list))
years

'2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025'

In [11]:
# Forming the final API to get data between 2011 - 2025
FINAL_API = BASE_API+indicator+"?periods="+years
FINAL_API

'https://www.imf.org/external/datamapper/api/v1/NGDPD?periods=2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025'

In [19]:
# Sending HTTP GET request
r = requests.get(FINAL_API)

status_code 200 denotes success

In [18]:
# Checking for the Status code
if r.status_code == 200:
    print("Success!!!!")
elif r.status_code == 404:
    print("Not Found!!!!")

Success!!!!


In [22]:
r.headers["Content-Type"]

'application/json'

In [23]:
# Data returned by the API
r.json()

{'values': {'NGDPD': {'SDN': {'2011': 66.448,
    '2012': 48.948,
    '2013': 52.892,
    '2014': 60.726,
    '2015': 64.534,
    '2016': 64.888,
    '2017': 48.906,
    '2018': 33.59,
    '2019': 31.484,
    '2020': 35.237,
    '2021': 35.141,
    '2022': 33.5,
    '2023': 36.983,
    '2024': 29.184,
    '2025': 35.897},
   'AFG': {'2011': 17.89,
    '2012': 20.293,
    '2013': 20.17,
    '2014': 20.616,
    '2015': 20.057,
    '2016': 18.02,
    '2017': 18.883,
    '2018': 18.336,
    '2019': 18.876,
    '2020': 20.136,
    '2021': 14.278,
    '2022': 14.501,
    '2023': 17.248,
    '2024': 18.08},
   'ALB': {'2011': 12.899,
    '2012': 12.324,
    '2013': 12.784,
    '2014': 13.246,
    '2015': 11.389,
    '2016': 11.862,
    '2017': 13.053,
    '2018': 15.38,
    '2019': 15.582,
    '2020': 15.271,
    '2021': 17.985,
    '2022': 19.189,
    '2023': 23.633,
    '2024': 27.084,
    '2025': 29.939},
   'DZA': {'2011': 218.332,
    '2012': 227.144,
    '2013': 229.701,
    '2014': 238